# Comparison of line search methods

In [1]:
import sys

sys.path.insert(1, "..")

import autoray as ar
from ttml.ttml import TTMLRegressor, TTMLClassifier, TTML
from ttml.tt_rlinesearch import nonmonotone_armijo, TTLS
from ttml.utils import *
from ttml.tensor_train import TensorTrain
import os.path
import pandas as pd
import numpy as np
import scipy.io.arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.kernel_approximation import RBFSampler, PolynomialCountSketch, Nystroem
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import KernelPCA
from tqdm import tqdm
import matplotlib.pyplot as plt


DATASET_FOLDER = "../datasets/data/"

from datasets import load_data

dataset_name = "power_plant"

dataset = load_data.dataset_loaders[dataset_name](DATASET_FOLDER)
X = dataset["X"]
y = dataset["y"]
# X = np.random.normal(size=(10000, 10))
# y = np.mean(X, axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X.astype(float),
    y.astype(float),
    test_size=0.2,
    random_state=179,
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [16]:
import itertools
import collections
from datetime import datetime
loss_dict = collections.defaultdict(list)
time_dict = collections.defaultdict(list)
n_runs = 10
rank = 3
n_thresh = 10
ttml_list = [TTML.random_from_data(X_train, 2, 5) for _ in range(n_runs)]
cg_method = 'fr'
for memory, initial_stepsize_method, run in tqdm(list(itertools.product((1,3,5),["bb1", "bb2", "qopt", "scalar"],range(10)))):
    start_time = np.datetime64(datetime.now())
    ttml = ttml_list[run]
    name = f"{memory}-{initial_stepsize_method}"

    ttls = TTLS(
        ttml.tt,
        y_train,
        ttml.observed_indices(X_train),
        line_search_method="armijo",
        cg_method = cg_method,
        initial_stepsize_method=initial_stepsize_method,
        memory=memory,
    )
    time_hist = []
    for i in (range(80)):
        ttls.step()
        time_delta = np.datetime64(datetime.now()) - start_time
        time_hist.append(time_delta / np.timedelta64(1, "ms"))
    time_hist = np.array(time_hist) / 1000
    time_dict[(memory, initial_stepsize_method)].append(time_hist)
    loss_dict[(memory, initial_stepsize_method)].append(np.array(ttls.loss_history))
    #print(f"{name}: {ttls.grad_calls} grad calls, {ttls.func_calls} func calls")

 97%|█████████▋| 116/120 [01:53<00:03,  1.11it/s]../ttml/tt_rlinesearch.py:448: RuntimeWarning: Linesearch failed to converge. Probably convergence has been reached.
  warnings.warn(
100%|██████████| 120/120 [01:57<00:00,  1.02it/s]


In [17]:
plt.figure(figsize=(10,6))
plt.title(f"Average losses over {n_runs} for different line search settings, cg method '{cg_method}'\n for a rank {rank} TTML with {n_thresh} thresholds for {dataset_name} dataset")

colors = {'bb1':'k','bb2':'g','qopt':'r','scalar':'b'}
ls = {1:'-',3:'--',5:'-.'}
for (memory,initial_stepsize_method), losses in loss_dict.items():
    name = f"{memory}-{initial_stepsize_method}"
    losses = np.stack(losses)
    y = np.mean(losses,axis=0)
    times = np.stack(time_dict[(memory,initial_stepsize_method)])
    x = np.mean(times,axis=0)
    plt.plot(x,y, label=name, c=colors[initial_stepsize_method], ls=ls[memory])

plt.xlabel("time (s)")
plt.ylabel("loss value")
plt.legend()
plt.yscale("log")